In [ ]:
# %load load.py
import numpy as np
import pandas as pd
import geopandas as gp
import holoviews as hv
import geoviews as gv
from geoviews import opts

import utils
import importlib
importlib.reload(utils)

gv.extension('bokeh')
opts.defaults(
    opts.Polygons( tools=['hover'], width=600, height=500),
    opts.Points( tools=['hover'], width=600, height=500),
    opts.Overlay(width=600, height=500)
    )

In [208]:
import panel as pn
import param
pn.extension()
from datetime import datetime

Stack

# Keep

#### Footprints

In [169]:
from geoviews import tile_sources as gvts


:WMTS   [Longitude,Latitude]

In [212]:
footprints = gv.Path(gp.read_file('temp/chi_mic.geojson'))
footprints.options(alpha=0)*gvts.EsriReference

:Overlay
   .Path.I :Path   [Longitude,Latitude]
   .WMTS.I :WMTS   [Longitude,Latitude]

#### Roads

In [58]:
roads = gv.Path(gp.read_file('Road Network/RoadNetwork.geojson'))

In [61]:
road_opts = opts.Path(alpha=0.5, color='blue')
roads.options(road_opts)

:Path   [Longitude,Latitude]

# Keep

#### Make color span

In [3]:
def palette_plot(cmap_name, direction=1, provider='colorcet'):

    """
    Searching Holoviews colormap listings by name and provider
    Outputs colorbar
    Specify reverse by direction = -1
    """
    from holoviews.plotting.util import process_cmap

    cmap = process_cmap(cmap=cmap_name, provider=provider)[::direction]
#     return cmbar(cmap).relabel(label=cmap_name)
    return cmap


def cmbar(cmap_hex_list):
    
    """
    Produces colorbar for list of hex colors
    """
    
    import numpy as np
    import holoviews as hv
    
    spacing = np.linspace(0, 1, len(cmap_hex_list))[np.newaxis]
    img_opts = opts.Image(cmap=cmap_hex_list, xaxis='bare', yaxis='bare', xticks=0, yticks=0, toolbar=None,\
                         frame_height=100, frame_width=400)
    return hv.Image(spacing, ydensity=1).opts(img_opts)

In [4]:
def partitioned(basemap, span=50, dl=0.5):
    colrs = []
    for b in basemap:
        addmap = color_fader(adjust_lightness(b, amount=1-dl), b, span) +\
                           color_fader(b, adjust_lightness(b, amount=1+dl),span)
        colrs.extend(addmap)
    return colrs

In [5]:
def color_fader(c1,c2,n): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    import matplotlib as mpl
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return [mpl.colors.to_hex((1-x)*c1 + x*c2) for x in np.linspace(0,1,n)]

In [6]:
def increase(cmap, slc, n_between, n_plus=5, dl=0.5):
    nslices = int(len(cmap)/slc)
    divisions = [cmap[slc*i:slc+slc*i] for i in range(nslices)]
    new_cmap = []
    for d in divisions:
        new_cmap.extend(color_fader(d[0],d[-1],n_between))
    new_cmap = color_fader(adjust_lightness(new_cmap[0], amount=1-dl), new_cmap[0], slc*n_plus) + new_cmap +\
               color_fader(new_cmap[-1], adjust_lightness(new_cmap[-1], amount=1-dl),slc*n_plus)
    return new_cmap

In [7]:
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return mc.to_hex(colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2]))

In [389]:
cmap = palette_plot('Spectral',provider='matplotlib')
(cmbar(cmap) + cmbar(partitioned(cmap[::16],80,0.6))+cmbar(increase(cmap,16, 50))\
+cmbar(cmap[::24])).cols(1)

:Layout
   .Image.I   :Image   [x,y]   (z)
   .Image.II  :Image   [x,y]   (z)
   .Image.III :Image   [x,y]   (z)
   .Image.IV  :Image   [x,y]   (z)

#### Colorbar normalization

In [134]:
def get_cmap_dct(chunks, param, cmap=None, mode='range', darken=0.5, lighten=0.5):
    
#     print(param)
    rng_dct = {}
    minmax = []
    for i,j in chunks.items():
        rng_dct[i] = {}
        seas_max = j[param].max()
        seas_min = j[param].min()
        rng_dct[i]['values'] = (seas_min, seas_max)
        minmax.extend([seas_min, seas_max])
    rng_dct['total'] = {}
    rng_dct['total']['values'] =  min(minmax), max(minmax)
    
    tot = rng_dct['total']['values']
    def slice_cmap(lo,hi, mode=mode, check_total=None):
        get_index = lambda x:int(round(len(cmap)*(x-tot[0])/(tot[1]-tot[0])))
        if mode =='range':
#             print(check_total)
            lo_index, hi_index = get_index(lo), get_index(hi)
            return cmap[lo_index:hi_index]
        elif mode == 'focus':
            ind = get_index(int(round((lo+hi)/2)))
            if ind == len(cmap):
                ind -= 1
#             print(ind, len(cmap))
            base = cmap[ind]
            return color_fader(adjust_lightness(base, amount=1-darken), base, 50) +\
                   color_fader(base, adjust_lightness(base, amount=1+lighten),50)    
    
    tot_colors = []
    for i,j in rng_dct.items():
        if i == 'total':
            rng_dct[i]['colors'] = tot_colors
        else:
            add_colors = slice_cmap(*j['values'], mode=mode)
            rng_dct[i]['colors'] = add_colors
            tot_colors.extend(add_colors)
    
    return rng_dct

In [9]:
cmap = palette_plot('Spectral',provider='matplotlib')
# new_cmap = partitioned(cmap[::16],80,0.6)
new_cmap = cmap[::24]
cmbar(new_cmap)

:Image   [x,y]   (z)

In [78]:
seasons = ['winter','spring','summer','fall']
files = [pd.read_csv(f'temp/avgs/{i}.csv') for i in seasons]
chunks = {s:f.groupby(['Lat','Lon']).mean() for s,f in zip(seasons,files)}
ranges = get_cmap_dct(chunks, new_cmap, darken=0.5, lighten=2, mode='focus')

Get maxes for temp across all seasons

In [59]:
chunks['spring'].columns

Index(['temp_K', 'dewpt_K', 'RH_pct', 'pres_Pa', 'RadDir_Wm_2', '_RadDif_Wm_2',
       'Longwave_Wm_2', 'Shortwave_Wm_2', 'WindDir_deg', 'WindSpd_ms_1'],
      dtype='object')

In [229]:
def grid_data(season, chunk, variables, ranges):
    var_plots = []
    
    ### Set Options ###
    
#     cmap = palette_plot('Reds',provider='matplotlib')
    cmap = ranges[season]['colors']
#     display(cmbar(cmap))
    plotter = 'bokeh'
#     hv.extension(plotter)
    
    if plotter == 'matplotlib':
        plotter_opts = opts.Points(vmin=100, vmax=300)#(width=200, height=200)
    else:
        #ht = 200
        #frame_width=int(ht*.575), frame_height=ht
        plotter_opts = opts.Points(frame_width=200, frame_height=200, alpha=1, padding=(0,0,0))
    
    gen_opts = [plotter_opts, 
                opts.Points(colorbar=True, toolbar='above', xaxis=None, yaxis=None, 
                            size=13, cmap=cmap, marker='s', axiswise=True)]
    for v in variables:
        
        ### Normalize color bar
        print(ranges[season]['values'])
        chunk = chunk.rename(columns={v:'temp'})
        title = season + ': ' + v.split('_')[0]
        var_plot = gv.Points(chunk, ['Lon', 'Lat'], vdims='temp').opts(title=title)#season)
        var_plot = var_plot.redim(temp=dict(range=ranges[season]['values']))
        var_plot.opts(gen_opts).opts(clabel='W/m2', color_index='temp') 
        var_plot.redim(temp=v)              
        chunk = chunk.rename(columns={'temp':v})
        output = var_plot*footprints.opts(alpha=0.3, color='black')
#         display(output)
        var_plots.append(output)
        
    return hv.Layout(var_plots).opts( toolbar=None, shared_axes=False)

view = hv.Layout([grid_data(i, chunks[i], ['temp_K'], ranges) for i in seasons])
view

TypeError: list indices must be integers or slices, not str

In [155]:
cmap = palette_plot('Fire',provider='matplotlib')
# new_cmap = partitioned(cmap[::16],80,0.6)
new_cmap = cmap[120::24]
cmbar(new_cmap)

:Image   [x,y]   (z)

In [154]:
cmap = palette_plot('Spectral',provider='matplotlib')
# new_cmap = partitioned(cmap[::16],80,0.6)
new_cmap = cmap[::70]
cmbar(new_cmap)


:Image   [x,y]   (z)

In [235]:
params = ['temp_K', 'dewpt_K', 'RH_pct', 'pres_Pa', 'RadDir_Wm_2', '_RadDif_Wm_2',
       'Longwave_Wm_2', 'Shortwave_Wm_2', 'WindDir_deg',]# 'WindSpd_ms_1']
params = [       'Longwave_Wm_2', 'Shortwave_Wm_2']
ranges = [get_cmap_dct(chunks, i, new_cmap, darken=0.7, lighten=2, mode='focus') for i in params]
views = [
    pn.Column(hv.Layout([grid_data(i, chunks[i], [j], r) for i in seasons]).opts(toolbar=None).cols(2)) for j,r in zip(params,ranges)]

(233.12052526765035, 234.32295421223958)
(334.6202377516741, 335.98888584728417)
(368.26990243647884, 370.21387561662584)
(293.0363903191598, 294.50538440881155)
(119.57774392766204, 121.34706144516778)
(232.98396437425595, 238.86394086142113)
(225.93519128308824, 240.91491662561273)
(108.65388386953552, 110.70356698668027)


In [ ]:
# MAKE DIFFS EQUAL ON COLORBARS

In [166]:
gvts

NameError: name 'gvts' is not defined

In [237]:
pn.Column(views[1]).save('shortwave.png')

In [231]:
for i in views:
#     i.save('test.png')
    display(i)

Column
    [0] HoloViews(Layout)

Column
    [0] HoloViews(Layout)

In [103]:
a = view.Overlay.I.Points.I

In [110]:
a.range('Lat'), a.range('Lon')

((41.858452, 41.891693), (-87.641479, -87.617188))

In [132]:
cmap = palette_plot('Reds',provider='matplotlib')
cmbar(cmap)

:Image   [x,y]   (z)

In [176]:
weath = view.Overlay.I.Points.I
rd = view.Overlay.I.Path.I

In [203]:
(\
weath.options(frame_width=275, frame_height=275, size=4, cmap=cmap) *\
#rd.options(frame_width=150, frame_height=275, color='black', alpha=0.2) *\
#bldgs.options(size=1, color='green', alpha=03.5)) *\
footprints.options(alpha=0.3, color='black')*\
gvts.Wikipedia.opts(padding=(0,0), alpha=0.5).redim(Longitude=hv.Dimension('Longitude', range=(-10, 90))))
# .redim(Lat=hv.Dimension('Lat', range=(41.858452, 41.891693)))\
# .redim(Lon=hv.Dimension('Lon', range=(-87.641479, -87.617188))) )
# view.Overlay.I.Points.I.options(frame_width=150, frame_height=275, size=10)

:Overlay
   .Points.I :Points   [Lon,Lat]   (temp)
   .Path.I   :Path   [Longitude,Latitude]
   .WMTS.I   :WMTS   [Longitude,Latitude]

In [182]:
p = gvts.CartoLight

In [184]:
p.dimensions()

[Dimension('Longitude'), Dimension('Latitude')]

In [180]:
hv.help(hv.Tiles)

Tiles

Online example: http://holoviews.org/reference/elements/bokeh/Tiles.html

-------------
Style Options
-------------

	alpha, level, max_zoom, min_zoom, render_parents, smoothing

(Consult bokeh's documentation for more information.)

------------
Plot Options
------------

The plot options are the parameters of the plotting class:

Parameters of 'TilePlot'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name                                 Value                         Type         Bounds     Mode  

active_tools                           []                          List       (0, None)    V RW  
align                                 None                    ObjectSelector               V RW  
apply_extents                         True                       Boolean        (0, 1)     V RW  
apply_ranges                          True                       Boolea

#### Colorbar ex.

In [22]:
data = utils.plot_bldgs().dframe()
data.head(1)

,Lon,Lat,area_total,MEAN_AVGHT
0,-87.628662,41.867863,285.41,1.74


In [34]:
plot = gv.Points(data, ['Lon', 'Lat'], ['MEAN_AVGHT', 'area_total']).redim(MEAN_AVGHT="x").redim(MEAN_AVGHT=dict(range=(0, 4000)))
new_opts = opts.Points(width=300, height=300, 
                        colorbar=True, toolbar='above', xaxis=None, yaxis=None, 
                        color_index=2, #size_index, 
                        size=3, cmap='fire')

plot.opts(new_opts).redim(
                        Lon="Longitude",
                        Lat="Latitude",
                        x='Ht', 
                        area_total='area'
                        )

:Points   [Longitude,Latitude]   (Ht,area)

#### Main

In [30]:
def get_chunks():
    
    iterator = utils.get_data('weather', old_weather_read=True)
    chunk_dict = {}
    j = 1
    for chunk in iterator:
        if j < 200:
            day, fine = chunk.time.iloc[0].split('_')
            hour, minute = fine.split(':')[:2]
            day = datetime.strptime(day, '%Y-%m-%d').date()
            hour = datetime.strptime(hour, '%H').time()
            minute = datetime.strptime(minute, '%M').time()
            chunk_dict[(day, hour, minute)] = chunk
        else:
            break
        j = j+1
    return chunk_dict

In [10]:
import warnings
warnings.filterwarnings('ignore')

class Weather(param.Parameterized):
    
    # select number of curves and max pks/curve for sample data
    chunk_dict = get_chunks()
    days = sorted(set([i[0] for i in chunk_dict.keys()]))
#     print(set(days))
#     print(days[-1])
    day = param.Date(default=days[0], bounds=(days[0], days[-1]))
    hour = param.Number(bounds=[0,23], step=1)
    minute = param.Number(bounds=[0,45], step=15)
    plots = None
    map_tile = utils.tile()#gv.tile_sources.StamenTerrain # EsriImagery
    
    path = 'temp/chi_mic.geojson'
    fp_df = gp.read_file(path)
    fp = utils.plot_footprints(fp_df)


    def plot(self):
        def grid_data(chunk, variables):
            var_plots = []
            plotter = 'bokeh'
            hv.extension(plotter)
            for v in variables:
                if plotter == 'matplotlib':
                    plotter_opts = opts.Points(vmin=100, vmax=300)#(width=200, height=200)
                else:
                    ht = 200
                    plotter_opts = opts.Points(frame_width=int(ht*.575), frame_height=ht, size=9, alpha=0.5, padding=(0,0,0))
#                 var_opts = opts.Points(cmap = 'bmw', color=v, colorbar=True, clabel=v, marker='s', xaxis='bare', yaxis='bare')
                var_opts = opts.Points(width=300, height=300, 
                                        colorbar=True, toolbar='above', xaxis=None, yaxis=None, 
                                        color_index=2, #size_index, 
                                        size=9, cmap='fire', clabel=v, marker='s')
#                 display(chunk.head())
#                 print()
                chunk = chunk.rename(columns={v:'temp'})
#                 display(chunk.head())
                var_plot = gv.Points(chunk, ['lon', 'lat'], vdims='temp')
                var_plot = var_plot.redim(temp=dict(range=(268, 268.3)))
                var_plot.opts([var_opts, plotter_opts])#*self.fp.opts(alpha=0.3)#*self.map_tile
                var_plot.redim(temp=v)              
                chunk = chunk.rename(columns={'temp':v})

                var_plots.append(var_plot)
            lay_opts = opts.Layout(normalize=True, toolbar=None)
            return hv.Layout(var_plots).opts(lay_opts)
        redate = lambda x,y : datetime.strptime(str(int(x)).zfill(2), f'%{y}').time()
#         print(datetime.strptime(self.day, '%Y-%m-%d').date())
#         print(redate(self.minute, 'M'))
        date_time = self.day, redate(self.hour, 'H'), redate(self.minute, 'M')
    
        try:
            dfs = self.chunk_dict[date_time]
            variables = ['temp_K', 'dewpt_K']#, 'pres_Pa', 'RH_pct', 'RadDir_Wm-2', ' RadDif_Wm-2', 'Longwave_Wm-2',
                        # 'ShortwaveNorm_Wm-2', 'Shortwave_Wm-2', 'WindDir_deg', 'WindSpd_ms-1', 'RainDpth_mm']
            self.plots = grid_data(dfs, variables)
        except KeyError:
            return 'No available data for this data and time.'
        
        return self.plots
    
    @param.depends('day','hour','minute')
    def view(self):
        print(self.param.day)
        day_widg =  pn.Param( self.param.day, widgets={'day':{'type':pn.widgets.DateSlider}} )
        return pn.Column(pn.Row(day_widg, self.param.hour, self.param.minute), self.plot)

    def animate(self):
        import time
        interv = 0.05
        hours = np.arange(0,23)
        mins = [0,15, 30, 45]
        for d in self.days:
            self.day = d
            time.sleep(interv)
            for h in hours:
                self.hour = h
                time.sleep(interv)
                for m in mins:
                    self.minute = m
                    time.sleep(interv)

w = Weather()
w.view()

AttributeError: 'Timestamp' object has no attribute 'split'

In [58]:
%%capture
w.animate()

In [12]:
def grid_data(chunk, variable):
    map_tile = gv.tile_sources.StamenTerrain # EsriImagery
    points = gv.Points(chunk, ['lon', 'lat'], vdims=variable)
    Visual = (points*map_tile).opts(opts.Points(cmap = 'bmw', color=variable, colorbar=True, clabel='Temp. K', width=600, height=600, marker='s'))
    #display(Visual)
    return Visual

chunk_size = 880
j = 0
temp_dict = {}
for chunk in utils.get_data('weather'):
    if j < 4:
        time = chunk.time[j*880].split('_')[1]
        temp_dict[time] = grid_data(chunk, 'temp_K')
    else:
        break
    j = j+1

print(temp_dict)
hmap = gv.HoloMap(temp_dict, kdims='time')
print(hmap)
hmap

{'00:00:00': :Overlay
   .Points.I :Points   [lon,lat]   (temp_K)
   .WMTS.I   :WMTS   [Longitude,Latitude], '00:15:00': :Overlay
   .Points.I :Points   [lon,lat]   (temp_K)
   .WMTS.I   :WMTS   [Longitude,Latitude], '00:30:00': :Overlay
   .Points.I :Points   [lon,lat]   (temp_K)
   .WMTS.I   :WMTS   [Longitude,Latitude], '00:45:00': :Overlay
   .Points.I :Points   [lon,lat]   (temp_K)
   .WMTS.I   :WMTS   [Longitude,Latitude]}
:HoloMap   [time]
   :Overlay
      .Points.I :Points   [lon,lat]   (temp_K)
      .WMTS.I   :WMTS   [Longitude,Latitude]


:HoloMap   [time]
   :Overlay
      .Points.I :Points   [lon,lat]   (temp_K)
      .WMTS.I   :WMTS   [Longitude,Latitude]